In [5]:
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
from torchvision import transforms
import skimage.measure as skm

global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SunspotSegmenter(nn.Module):
    def __init__(self):
        super(SunspotSegmenter, self).__init__()
        # Encoder - extracts features
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Decoder that will generates segmentation mask
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.dec1 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.dec2 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 1, kernel_size=1),
            nn.Sigmoid()  # Output between 0 and 1, good for binary segmentation
        )

    def forward(self, x):
        # Two reducing layers
        x1 = self.enc1(x)
        x = self.pool1(x1)
        
        x2 = self.enc2(x)
        x = self.pool2(x2)
        
        # Then decoding layers
        x = self.upsample1(x)
        x = self.dec1(x)
        
        x = self.upsample2(x)
        x = self.dec2(x)
        
        return x

In [ ]:
def detect_sunspots(model, input_path, device, threshold=0.5):
    image_name = input_path.split("\\")[-1] 
    image_name = image_name.split('/')[-1] #get what is after the last /
    image_name = image_name.split('.')[0]


    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    image = Image.open(input_path).convert('RGB')
    original = image.copy()
    image = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        mask = model(image) #WILL RETURN A "PROBABILISTIC" MASK
            
        # Resize mask to match original image size (1024x1024)
        mask_resized = cv2.resize(
        mask.astype(np.float32), 
        (image.shape[1], image.shape[0]),
        interpolation=cv2.INTER_NEAREST
    )

        mask = (mask_resized > threshold).float()
        mask = mask.cpu().squeeze().numpy()
    
    return {
        'original': original,
        'mask': mask,
        'image_name': image_name
    }


def analyze_sunspots(results, min_area=7, output_folder='results'):    
    # Get mask and original image
    mask = results['mask']
    original_image = np.array(results['original'])
    
    ## THIS FUNCTION IS BASICALLY THE SAME AS METHOD 1

    # Get connected components
    labels = skm.label(mask, connectivity=2, background=0)
    regionprops = skm.regionprops(labels)
    
    # Filter regions
    regionprops = [prop for prop in regionprops if 
                   10000 >= prop.area >= min_area]
    
    # Create border mask
    border_mask = np.zeros_like(original_image)
    
    # Draw borders
    for region in regionprops:
        filled_area = region.filled_image
        minr, minc, maxr, maxc = region.bbox
        
        # Create temporary mask
        temp_mask = np.zeros((maxr - minr, maxc - minc), dtype=bool)
        temp_mask[0:filled_area.shape[0], 0:filled_area.shape[1]] = filled_area
        
        # Find boundary pixels
        for x in range(temp_mask.shape[0]):
            for y in range(temp_mask.shape[1]):
                if temp_mask[x, y]:
                    # Check boundary condition
                    if (x == 0 or x == temp_mask.shape[0]-1 or 
                        y == 0 or y == temp_mask.shape[1]-1 or
                        not temp_mask[x-1, y] or not temp_mask[x+1, y] or
                        not temp_mask[x, y-1] or not temp_mask[x, y+1]):
                        border_mask[x + minr, y + minc] = [255, 255, 255]
    
    # Superpose borders
    border_superposed = cv2.addWeighted(original_image, 1, border_mask, 1, 0)
    
    # Print analysis
    print(f'Number of sunspots: {len(regionprops)}')
    print('')
    for i, prop in enumerate(regionprops):
        print(f'Sunspot {i + 1}: {int(prop.area)} pixels')
    
    # Save results
    img_name = results['image_name']
    output_path = f'{output_folder}/output_{img_name}.jpg'
    cv2.imwrite(output_path, cv2.cvtColor(border_superposed, cv2.COLOR_RGB2BGR))
    print(f'Image with borders saved at {output_path}')

In [3]:
def main(input_path, min_area=7, threshold=0.5, output_folder='output'):
    # Load model
    model = SunspotSegmenter().to(device)
    model.load_state_dict(torch.load('new_model.pth'))
    
    # Detect sunspots
    results = detect_sunspots(model, input_path, device, threshold)
    
    # Analyze sunspots
    analyze_sunspots(results, min_area, output_folder)

In [4]:
main('../images/20250130_223000_1024_HMIIF.jpg')

C:\Users\guezz\AppData\Local\Temp/ipykernel_500/3160248553.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('new_model.pth'))


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>